### Modules and Config

In [5]:
# Installed
import numpy as np
import theano
import theano.tensor as TT
from pdb import set_trace
theano.config.optimizer='None'
theano.gof.cc.get_module_cache().clear()

# Local
import tools.shared as shared
import tools.misc as misc
reload(misc)
import tools.nef as nef 
reload(nef)
import tools.gru as gru
reload(gru)

<module 'tools.gru' from 'tools/gru.pyc'>

**Configuration**

In [6]:
# outer world
nr_words = 40     # Number of words
voc_size = 100    # Vocabulary size 
emb_size = 50     # Embedding size
nr_classes = 2    # Number of classes
# 
# Sketch matrix
sketch_dim = 10   # Sketch dimension
nr_sketch = 3     # Sketch number
# Attention
attention_dim = 6 # Attention dimension
conv_dim = 3      # Attention convolutionsl dimension 

# Train param
lrate = 1.0

### Forward pass

In [7]:
# Embeddings
E = shared.random(size=(emb_size, voc_size), name='embeddings')
# GRU on the source
param_GRU = gru.init_gru(emb_size, sketch_dim, bias=True)

In [8]:
# Input
x = TT.ivector('x')
# Embedding
emb = E[:, x]
# RNN
H = gru.gru(emb.T, param_GRU).T
# Easy first 
p_y = nef.easy_first(H, nr_sketch, attention_dim, sketch_dim, conv_dim, nr_classes)

In [9]:
param = misc.get_param(p_y, named_only=True)
param

[inference.w_p,
 gru.b_U,
 gru.U,
 gru.b_Ur,
 gru.U_r,
 gru.b_Uz,
 gru.U_z,
 gru.[b b_r b_z],
 gru.[W, W_r, W_z],
 embeddings,
 attention.W_sz,
 attention.W_hz,
 attention.v,
 sketch.W_s,
 sketch.W_ss,
 sketch.W_hs,
 inference.W_sp]

**Fake Example Forward**

In [10]:
# Fake data
data_x = np.random.randint(0, voc_size-1, size=(nr_words)).astype('int32')
dbg = theano.function([x], p_y)
print [dat.shape for dat in dbg(data_x)]

[(40,), (40,)]


### Training

In [11]:
# Cost for ground truth
y = TT.ivector('x')
cost = -TT.mean(TT.log(p_y)[y, TT.arange(y.shape[0])])

**Fake Example Cost**

In [12]:
# Fake data
data_x = np.random.randint(0, voc_size-1, size=(nr_words)).astype('int32')
data_y = np.random.randint(0, nr_classes-1, size=(nr_words)).astype('int32')
dbg = theano.function([x, y], cost)
print dbg(data_x, data_y)

0.693147199973


In [13]:
updates = [(par, par - lrate*TT.grad(cost, par)) for par in param]
batch_train = theano.function([x, y], cost, updates=updates)

**Fake Example Batch Update**

In [14]:
batch_train(data_x, data_y)

array(0.6931471999725708)